# Semestrální projekt KGI/PRODA
 ## Převod obrazu na ASCII Art

Nejprve musíme importovat knihovny
PIL (Python Imaging Library):
- **Image:** Tato knihovna poskytuje schopnost otevírat, vytvářet a manipulovat s různými formáty obrázků. Ve vašem kódu se využívá k otevření obrázku, jeho zpracování a uložení.
- **ImageDraw:** Tato knihovna umožňuje kreslit na obrázek vytvořený pomocí PIL. Používá se k vykreslení textu na obrázek.
- **ImageFont:** Slouží k načítání fontů pro použití s PIL. Vaše aplikace používá tento modul k načtení fontu, který má být použit při vykreslování textu na obrázek.
- **math:**
Tato knihovna poskytuje matematické funkce. Ve vašem kódu se používá k výpočtu indexu znaku na základě jasu pixelu.
- **svgwrite:**
Jedná se o knihovnu pro generování SVG (Scalable Vector Graphics) v Pythonu. Ve vašem kódu je použita k vytvoření výsledného SVG souboru, který obsahuje textové znaky v závislosti na jasu pixelů.

In [174]:
from PIL import Image, ImageDraw, ImageFont
import math
import svgwrite

Seznam znaků, které budou použity pro reprezentaci intenzity šedi (převod na šedou barvu je jednodušší než převod na RGB)


In [175]:
chars = "NaČelníSkloPadajíKapkyDěště"[::-1]

Převod řetězce na pole znaků a počtu znaků v seznamu

In [176]:
charArray = list(chars)
charLength = len(charArray)

Nyní je nutné stanovit interval pro trasování intenzity šedi na znaky

In [177]:
interval = charLength / 256

### Změna velikostí
- scaleFactor:
Tato proměnná určuje poměr, o který se má zmenšit původní obrázek před jeho zpracováním na textové umění. Ve vašem kódu je nastavena na hodnotu 0.4, což znamená, že vstupní obrázek bude zmenšen na 40 % své původní velikosti před konverzí na text.
- oneCharWidth:
Tato proměnná určuje šířku jednoho znaku v textovém umění, který bude reprezentovat jeden pixel v původním obrázku. Ve vašem kódu je nastavena na hodnotu 8, což znamená, že každý pixel v obrázku bude reprezentován osmi znaky širokým textem.
- oneCharHeight:
Tato proměnná určuje výšku jednoho znaku v textovém umění. Ve vašem kódu je nastavena na hodnotu 13, což znamená, že každý pixel v obrázku bude reprezentován třinácti znaky vysokým textem.

In [178]:
scaleFactor = 0.4
oneCharWidth = 8
oneCharHeight = 13

Získání znaku na základě intenzity šedi

In [179]:
def getChar(inputInt):
    return charArray[math.floor(inputInt * interval)]

### Zdrojový obraz pro celý script
- (.png)

In [180]:
im = Image.open("dog.png")

### Načtení fontu. 
- Je možné vložit jakýkoliv font, pokud se žádný nevybere a tento kód bude smazán, script bude přesto fungovat.

In [181]:

fnt = ImageFont.truetype('C:/Windows/Fonts/Arial.ttf', 9)

Změna velikosti obrázku

In [182]:

width, height = im.size
im = im.resize((int(scaleFactor * width), int(scaleFactor * height * (oneCharWidth / oneCharHeight))), Image.NEAREST)
width, height = im.size
pix = im.load()

Vytvoření nového obrazu

In [183]:
outputImage = Image.new('RGB', (oneCharWidth * width, oneCharHeight * height), color=(0, 0, 0))
d = ImageDraw.Draw(outputImage)

Vytvoření SVG souboru

In [184]:

svg_output = svgwrite.Drawing(filename='pes_vykres.svg', size=(oneCharWidth * width, oneCharHeight * height))

Nyní se prochází každý jednotlivý pixel obrázku pro přípravu nových souborů

In [185]:
for i in range(height):
    for j in range(width):
        r, g, b = pix[j, i]
        h = int(r / 3 + g / 3 + b / 3)
        pix[j, i] = (h, h, h)
        d.text((j * oneCharWidth, i * oneCharHeight), getChar(h), font=fnt, fill=(r, g, b))
        svg_output.add(svg_output.text(getChar(h), insert=(j * oneCharWidth, i * oneCharHeight),
                                       fill=f'rgb({r}, {g}, {b})', font_size=9, font_family="Arial"))

Uložení obrazu + možnost zadat v jaké kvalitě (dpi)

In [186]:
outputImage.save('pes_obr.png', dpi=(300, 300))

Uložení SVG

In [187]:
svg_output.save()